In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#from yoloface import face_analysis
import numpy
import cv2

In [3]:
%pip install numpy --upgrade

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: C:\Users\dharu\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, LSTM, Dense, concatenate
from tensorflow.keras.models import Model

In [5]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.models as models
import torch.optim as optim

C:\Users\dharu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
datagen = ImageDataGenerator(
    rescale=1.0 / 255,  
    validation_split=0.2 
)

In [7]:
train_generator = datagen.flow_from_directory(
    r"C:\Users\dharu\OneDrive\Desktop\6th Semester\Cryptography\lfw-funneled\lfw_funneled",
    target_size=(224, 224),  
    batch_size=32,
    class_mode='categorical',  
    subset='training'
)

Found 12432 images belonging to 5749 classes.


In [8]:
validation_generator = datagen.flow_from_directory(
    r"C:\Users\dharu\OneDrive\Desktop\6th Semester\Cryptography\lfw-funneled\lfw_funneled",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 1137 images belonging to 5749 classes.


In [9]:
from tensorflow.keras.applications import ResNet50
import os

In [10]:
original_images_folder = r"C:\Users\dharu\OneDrive\Desktop\6th Semester\Cryptography\lfw-funneled\lfw_funneled"

In [11]:
file_names = os.listdir(original_images_folder)

In [12]:
file_names

['Aaron_Eckhart',
 'Aaron_Guiel',
 'Aaron_Patterson',
 'Aaron_Peirsol',
 'Aaron_Pena',
 'Aaron_Sorkin',
 'Aaron_Tippin',
 'Abbas_Kiarostami',
 'Abba_Eban',
 'Abdel_Aziz_Al-Hakim',
 'Abdel_Madi_Shabneh',
 'Abdel_Nasser_Assidi',
 'Abdoulaye_Wade',
 'Abdulaziz_Kamilov',
 'Abdullah',
 'Abdullah_Ahmad_Badawi',
 'Abdullah_al-Attiyah',
 'Abdullah_Gul',
 'Abdullah_Nasseef',
 'Abdullatif_Sener',
 'Abdul_Majeed_Shobokshi',
 'Abdul_Rahman',
 'Abel_Aguilar',
 'Abel_Pacheco',
 'Abid_Hamid_Mahmud_Al-Tikriti',
 'Abner_Martinez',
 'Abraham_Foxman',
 'Aby_Har-Even',
 'Adam_Ant',
 'Adam_Freier',
 'Adam_Herbert',
 'Adam_Kennedy',
 'Adam_Mair',
 'Adam_Rich',
 'Adam_Sandler',
 'Adam_Scott',
 'Adelina_Avila',
 'Adel_Al-Jubeir',
 'Adisai_Bodharamik',
 'Adolfo_Aguilar_Zinser',
 'Adolfo_Rodriguez_Saa',
 'Adoor_Gopalakarishnan',
 'Adriana_Lima',
 'Adriana_Perez_Navarro',
 'Adrianna_Zuzic',
 'Adrian_Annus',
 'Adrian_Fernandez',
 'Adrian_McPherson',
 'Adrian_Murrell',
 'Adrian_Nastase',
 'Adrien_Brody',
 'Afton_S

In [13]:
from tensorflow.keras.applications import ResNet50,EfficientNetV2B1
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

In [14]:
base_model = ResNet50(weights='imagenet', include_top=False)

In [15]:
face_model = Sequential()
face_model.add(base_model)
face_model.add(GlobalAveragePooling2D())
face_model.add(Dense(256, activation='relu')) 
face_model.add(Dense(5749, activation='softmax'))

In [16]:
for layer in base_model.layers:
    layer.trainable = False

In [17]:
audio_dataset = r"C:\Users\dharu\OneDrive\Desktop\6th Semester\Cryptography\AudioDataset"

In [18]:
import os
audio_classes = os.listdir(audio_dataset)

In [20]:
import numpy as np
#from pyAudioAnalysis import ShortTermFeatures, MidTermFeatures
import matplotlib.pyplot as plt

In [21]:
!pip install matplotlib

In [25]:
!pip install yaafe

ERROR: Could not find a version that satisfies the requirement yaafe (from versions: none)
ERROR: No matching distribution found for yaafe


In [35]:
!pip install numpy==1.23.1

   ---------------------------------------- 14.7/14.7 MB 955.7 kB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
altair 5.2.0 requires pandas>=0.25, which is not installed.
altair 5.2.0 requires toolz, which is not installed.
audiosr 0.0.6 requires pandas, which is not installed.
audiosr 0.0.6 requires unidecode, which is not installed.
datasets 2.17.0 requires pandas, which is not installed.
formulaic 0.6.6 requires pandas>=1.0, which is not installed.
formulaic 0.6.6 requires wrapt>=1.0, which is not installed.
gradio 4.16.0 requires pandas<3.0,>=1.0, which is not installed.
lifelines 0.27.8 requires pandas>=1.0.0, which is not installed.
spacy 3.7.2 requires smart-open<7.0.0,>=5.2.1, which is not installed.
tensorboard 2.15.1 requires markdown>=2.6.8, which is not installed.
tensorboard 2.15.1 requires werkzeug>=1.0.1, which is not installed.
tensorflow-datasets 4.9.3 requires toml, which is not installed.
tensorflow-data

In [2]:
%pip install librosa

  Obtaining dependency information for librosa from https://files.pythonhosted.org/packages/e2/a2/4f639c1168d7aada749a896afb4892a831e2041bebdcf636aebfe9e86556/librosa-0.10.1-py3-none-any.whl.metadata
  Obtaining dependency information for audioread>=2.1.9 from https://files.pythonhosted.org/packages/57/8d/30aa32745af16af0a9a650115fbe81bde7c610ed5c21b381fca0196f3a7f/audioread-3.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for numba>=0.51.0 from https://files.pythonhosted.org/packages/0c/fc/aecc9db1cb2707cede48779a50a67fdee270dc171e833027e5afda747701/numba-0.59.1-cp310-cp310-win_amd64.whl.metadata
  Obtaining dependency information for soundfile>=0.12.1 from https://files.pythonhosted.org/packages/50/ff/26a4ee48d0b66625a4e4028a055b9f25bc9d7c7b2d17d21a45137621a50d/soundfile-0.12.1-py2.py3-none-win_amd64.whl.metadata
  Obtaining dependency information for pooch>=1.0 from https://files.pythonhosted.org/packages/f4/72/8ae0f1ba4ce6a4f6d4d01a60a9fdf690fde188c45c1872b0b4ddb0


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: C:\Users\dharu\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [21]:
import librosa

In [22]:
file = librosa.load(r"C:\Users\dharu\OneDrive\Desktop\NUSProject\classes\breathing\1-18631-A-23.wav")

In [23]:
datagen = ImageDataGenerator(
    rescale=1.0 / 255,  
    validation_split=0.2 
)

In [24]:
train_generator_sign = datagen.flow_from_directory(
    r"C:\Users\dharu\OneDrive\Desktop\6th Semester\Cryptography\SignatureDataset",
    target_size=(224, 224),  
    batch_size=32,
    class_mode='categorical',  
    subset='training'
)

Found 224913 images belonging to 5767 classes.


In [25]:
validation_generator_sign = datagen.flow_from_directory(
    r"C:\Users\dharu\OneDrive\Desktop\6th Semester\Cryptography\SignatureDataset",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 51903 images belonging to 5767 classes.


In [26]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
import os

In [27]:
base_model_vgg = VGG16(weights='imagenet', include_top=False)

In [28]:
model = Sequential()
model.add(base_model_vgg)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu')) 
model.add(Dense(len(os.listdir(r"C:\Users\dharu\OneDrive\Desktop\6th Semester\Cryptography\SignatureDataset")), activation='softmax'))

In [29]:
for layer in base_model_vgg.layers:
    layer.trainable = False

In [30]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [31]:
data_face = r"C:\Users\dharu\OneDrive\Desktop\6th Semester\Cryptography\model_face"

In [32]:
from sklearn.preprocessing import LabelEncoder
lbl = LabelEncoder()
labels = lbl.fit(os.listdir(data_face))
final = labels.transform(os.listdir(data_face))


In [35]:
dataset_face = ImageFolder(data_face, transform=transform)

In [36]:
train_ratio = 0.8 

In [37]:
train_size = int(train_ratio * len(dataset_face))
test_size = len(dataset_face) - train_size

In [38]:
from torch.utils.data import random_split
train_dataset_face, test_dataset_face = random_split(dataset_face, [train_size, test_size])

In [39]:
train_loader_face = DataLoader(train_dataset_face, batch_size=32, shuffle=True)
test_loader_face = DataLoader(test_dataset_face, batch_size=32)

In [40]:
data_signature = r"C:\Users\dharu\OneDrive\Desktop\6th Semester\Cryptography\model_sign"

In [41]:
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),          
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
])

In [42]:
dataset_signature = ImageFolder(data_signature, transform=transform)

In [43]:
train_dataset_signature, test_dataset_signature = random_split(dataset_signature, [train_size, test_size])

In [44]:
train_loader_sign = DataLoader(train_dataset_signature, batch_size=32, shuffle=True)
test_loader_sign = DataLoader(test_dataset_signature, batch_size=32)

In [45]:
face_model = Sequential()
face_model.add(base_model)
face_model.add(GlobalAveragePooling2D())
face_model.add(Dense(256, activation='relu')) 
face_model.add(Dense(5749, activation='softmax'))

In [46]:
face_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [47]:
base_model_eff = EfficientNetV2B1(weights='imagenet', include_top=False)

In [48]:
sign_model = Sequential()
sign_model.add(base_model_eff)
sign_model.add(GlobalAveragePooling2D())
sign_model.add(Dense(256, activation='relu')) 
sign_model.add(Dense(5749, activation='softmax'))

In [49]:
sign_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [50]:
import os
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder

class CombinedModel(nn.Module):
    def __init__(self, base_model_face, base_model_sign, face_model, sign_model):
        super(CombinedModel, self).__init__()
        self.base_model_face = base_model_face
        self.base_model_sign = base_model_sign
        self.face_model = face_model
        self.sign_model = sign_model
    
    def forward(self, x_face, x_sign):
        features_face = self.base_model_face(x_face)
        features_sign = self.base_model_sign(x_sign)
        face_output = self.face_model(features_face)
        sign_output = self.sign_model(features_sign)
        return face_output, sign_output

In [51]:
combined_model = CombinedModel(base_model, base_model_eff, face_model, sign_model)

In [52]:
import torch
import torch.nn as nn

In [53]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [54]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [55]:
dataset_face = ImageFolder(data_face, transform=transform)

In [56]:
dataset_signature = ImageFolder(data_signature, transform=transform)

In [57]:
train_dataset_face, test_dataset_face = random_split(dataset_face, [train_size, test_size])
train_dataset_signature, test_dataset_signature = random_split(dataset_signature, [train_size, test_size])


In [58]:
train_loader_face = DataLoader(train_dataset_face, batch_size=32, shuffle=True)
test_loader_face = DataLoader(test_dataset_face, batch_size=32)
train_loader_sign = DataLoader(train_dataset_signature, batch_size=32, shuffle=True)
test_loader_sign = DataLoader(test_dataset_signature, batch_size=32)

In [59]:
shared_conv = nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2)
)


In [60]:
face_branch = nn.Sequential(
    shared_conv,
    nn.Flatten(),
    nn.Linear(128 * 56 * 56, 256),  
    nn.ReLU(),
    nn.Linear(256, len(os.listdir(data_face)))  
)

In [61]:
signature_branch = nn.Sequential(
    shared_conv,
    nn.Flatten(),
    nn.Linear(128 * 56 * 56, 256),  
    nn.ReLU(),
    nn.Linear(256, len(os.listdir(data_signature)))  
)


In [62]:
combined_model = nn.ModuleDict({
    'face_branch': face_branch,
    'signature_branch': signature_branch
})

In [63]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(combined_model.parameters(), lr=0.001)

In [64]:
num_epochs = 1

In [71]:
#inputs_face, labels_face = inputs_face.to(device), labels_face.to(device)
#inputs_sign, labels_sign = inputs_sign.to(device), labels_sign.to(device)

In [66]:
for epoch in range(num_epochs):
    combined_model.train()

    running_loss = 0.0
    
    for data_face, data_sign in zip(train_loader_face, train_loader_sign):
        
        inputs_face, labels_face = data_face
        inputs_sign, labels_sign = data_sign
        inputs_face, labels_face = inputs_face.to(device), labels_face.to(device)
        inputs_sign, labels_sign = inputs_sign.to(device), labels_sign.to(device)
        
        optimizer.zero_grad()
        
        outputs_face = combined_model['face_branch'](inputs_face)
        
        outputs_sign = combined_model['signature_branch'](inputs_sign)
        
        loss_face = criterion(outputs_face, labels_face)
       
        loss_sign = criterion(outputs_sign, labels_sign)
        
        loss = loss_face + loss_sign
       
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs_face.size(0)
    
    epoch_loss = running_loss / len(train_loader_face.dataset)
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

Epoch [1/1], Loss: 41.7102


In [69]:
from torchsummary import summary
print("Face Branch:")
summary(combined_model['face_branch'], input_size=(3, 224, 224))

Face Branch:
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
         MaxPool2d-3         [-1, 64, 112, 112]               0
            Conv2d-4        [-1, 128, 112, 112]          73,856
              ReLU-5        [-1, 128, 112, 112]               0
         MaxPool2d-6          [-1, 128, 56, 56]               0
           Flatten-7               [-1, 401408]               0
            Linear-8                  [-1, 256]     102,760,704
              ReLU-9                  [-1, 256]               0
           Linear-10                    [-1, 7]           1,799
Total params: 102,838,151
Trainable params: 102,838,151
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.57
Forward/backward pass size (MB): 85.75
Params size 

In [70]:
print("Signature Branch:")
summary(combined_model['signature_branch'], input_size=(3, 224, 224))

Signature Branch:
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
         MaxPool2d-3         [-1, 64, 112, 112]               0
            Conv2d-4        [-1, 128, 112, 112]          73,856
              ReLU-5        [-1, 128, 112, 112]               0
         MaxPool2d-6          [-1, 128, 56, 56]               0
           Flatten-7               [-1, 401408]               0
            Linear-8                  [-1, 256]     102,760,704
              ReLU-9                  [-1, 256]               0
           Linear-10                    [-1, 7]           1,799
Total params: 102,838,151
Trainable params: 102,838,151
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.57
Forward/backward pass size (MB): 85.75
Params 

In [72]:
print("Shared Convulutional Layer")
summary(shared_conv, input_size=(3, 224, 224))

Shared Convulutional Layer
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
         MaxPool2d-3         [-1, 64, 112, 112]               0
            Conv2d-4        [-1, 128, 112, 112]          73,856
              ReLU-5        [-1, 128, 112, 112]               0
         MaxPool2d-6          [-1, 128, 56, 56]               0
Total params: 75,648
Trainable params: 75,648
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.57
Forward/backward pass size (MB): 82.69
Params size (MB): 0.29
Estimated Total Size (MB): 83.55
----------------------------------------------------------------
